In [4]:
from __future__ import annotations

import asyncio
import nest_asyncio

# make asyncio.run() works in notebook
nest_asyncio.apply()

In [8]:
from metagpt.config2 import Config
from metagpt.provider.llm_provider_registry import create_llm_instance
from metagpt.utils.cost_manager import CostManager

cfg = Config.default()

llm = create_llm_instance(cfg.llm)
# llm.cost_manager = CostManager()
llm

In [7]:
asyncio.run(llm.aask("你好"))

Of course, I'm here to help. How can I assist you

2024-03-16 23:03:23.170 | INFO     | metagpt.utils.cost_manager:update_cost:57 - Total running cost: $0.000 | Max budget: $10.000 | Current cost: $0.000, prompt_tokens: 14, completion_tokens: 18


 today?


"Of course, I'm here to help. How can I assist you today?"

In [11]:
from pydantic import Field, Union, Literal

from metagpt.logs import logger
from metagpt.roles import Role
from metagpt.schema import Message, Task, TaskResult

from metagpt.actions.di.execute_nb_code import ExecuteNbCode
from metagpt.actions.di.write_analysis_code import CheckData, WriteAnalysisCode

In [3]:
class CodeInterpreter(Role):
    name: str = "OpenDevin"
    profile: str = "CodeInterpreter"
    auto_run: bool = True
    use_plan: bool = True
    use_reflection: bool = False
    execute_code: ExecuteNbCode = Field(default_factory=ExecuteNbCode, exclude=True)
    tools: Union[str, list[str]] = []  # Use special symbol ["<all>"] to indicate use of all registered tools
    react_mode: Literal["plan_and_act", "react"] = "plan_and_act"
    max_react_loop: int = 10  # used for react mode

    @property
    def working_memory(self):
        return self.rc.working_memory

    async def _think(self) -> bool:
        """Useful in 'react' mode. Use LLM to decide whether and what to do next."""
        user_requirement = self.get_memories()[0].content
        context = self.working_memory.get()

        if not context:
            # just started the run, we need action certainly
            self.working_memory.add(self.get_memories()[0])  # add user requirement to working memory
            self._set_state(0)
            return True

        prompt = REACT_THINK_PROMPT.format(user_requirement=user_requirement, context=context)
        rsp = await self.llm.aask(prompt)
        rsp_dict = json.loads(CodeParser.parse_code(block=None, text=rsp))
        self.working_memory.add(Message(content=rsp_dict["thoughts"], role="assistant"))
        need_action = rsp_dict["state"]
        self._set_state(0) if need_action else self._set_state(-1)

        return need_action

    async def _act(self) -> Message:
        """Useful in 'react' mode. Return a Message conforming to Role._act interface."""
        code, _, _ = await self._write_and_exec_code()
        return Message(content=code, role="assistant", cause_by=WriteAnalysisCode)

    async def _plan_and_act(self) -> Message:
        rsp = await super()._plan_and_act()
        await self.execute_code.terminate()
        return rsp

    async def _act_on_task(self, current_task: Task) -> TaskResult:
        """Useful in 'plan_and_act' mode. Wrap the output in a TaskResult for review and confirmation."""
        code, result, is_success = await self._write_and_exec_code()
        task_result = TaskResult(code=code, result=result, is_success=is_success)
        return task_result

    async def _write_and_exec_code(self, max_retry: int = 3):
        counter = 0
        success = False

        # plan info
        plan_status = self.planner.get_plan_status() if self.use_plan else ""

        # tool info
        if self.tools:
            context = (
                self.working_memory.get()[-1].content if self.working_memory.get() else ""
            )  # thoughts from _think stage in 'react' mode
            plan = self.planner.plan if self.use_plan else None
            tool_info = await self.tool_recommender.get_recommended_tool_info(context=context, plan=plan)
        else:
            tool_info = ""

        # data info
        await self._check_data()

        while not success and counter < max_retry:
            ### write code ###
            code, cause_by = await self._write_code(counter, plan_status, tool_info)

            self.working_memory.add(Message(content=code, role="assistant", cause_by=cause_by))

            ### execute code ###
            result, success = await self.execute_code.run(code)
            print(result)

            self.working_memory.add(Message(content=result, role="user", cause_by=ExecuteNbCode))

            ### process execution result ###
            counter += 1

            if not success and counter >= max_retry:
                logger.info("coding failed!")
                review, _ = await self.planner.ask_review(auto_run=False, trigger=ReviewConst.CODE_REVIEW_TRIGGER)
                if ReviewConst.CHANGE_WORDS[0] in review:
                    counter = 0  # redo the task again with help of human suggestions

        return code, result, success

    async def _write_code(
        self,
        counter: int,
        plan_status: str = "",
        tool_info: str = "",
    ):
        todo = self.rc.todo  # todo is WriteAnalysisCode
        logger.info(f"ready to {todo.name}")
        use_reflection = counter > 0 and self.use_reflection  # only use reflection after the first trial

        user_requirement = self.get_memories()[0].content

        code = await todo.run(
            user_requirement=user_requirement,
            plan_status=plan_status,
            tool_info=tool_info,
            working_memory=self.working_memory.get(),
            use_reflection=use_reflection,
        )

        return code, todo

    async def _check_data(self):
        if (
            not self.use_plan
            or not self.planner.plan.get_finished_tasks()
            or self.planner.plan.current_task.task_type
            not in [
                TaskType.DATA_PREPROCESS.type_name,
                TaskType.FEATURE_ENGINEERING.type_name,
                TaskType.MODEL_TRAIN.type_name,
            ]
        ):
            return
        logger.info("Check updated data")
        code = await CheckData().run(self.planner.plan)
        if not code.strip():
            return
        result, success = await self.execute_code.run(code)
        if success:
            print(result)
            data_info = DATA_INFO.format(info=result)
            self.working_memory.add(Message(content=data_info, role="user", cause_by=CheckData))

In [5]:
url = "https://pitchhub.36kr.com/financing-flash"
domain = "快讯"

prompt = f"""抓取{url}中'{domain}'的内容，并整理成markdown存档。

大概的流程：
- 使用工具抓取网页中的可见文本
- 提取网页文本中'{domain}'相关的内容。注意网页中可能包含导航，只需要抽取'{domain}'的具体内容
- 对抽取结果进行归类，并保存成markdown表格: {domain}.md
"""

ci = CodeInterpreter(use_tools=True, tools=["scrape_web_playwright"])
asyncio.run(ci.run(prompt))

2024-03-16 22:28:15.120 | INFO     | __main__:__init__:24 - will only use ['scrape_web_playwright'] as tools


Here is a plan to achieve the goal of extracting '快讯' content from the provided webpage and organizing it into a Markdown file:

```json
[
    {
        "task_id": "1",
        "dependent_task_ids": [],
        "instruction": "Use a web scraping tool to retrieve the visible text from the webpage."
    },
    {
        "task_id": "2",
        "dependent_task_ids": ["1"],
        "instruction": "Filter out the content related to '快讯' and exclude navigation or other irrelevant elements."
    },
    {
        "task_id": "3",
        "dependent_task_ids": ["2"],
        "instruction": "Categorize the extracted content and format it into a Markdown table."
    },
    {
        "task_id": "4",
        "dependent_task_ids": ["3"],
        "instruction": "Save the formatted content into a file named '快讯.md'."
    }
]
``` 

Each task is dependent on the previous one, ensuring that the process is followed step by step. Task 1 is to scrape the text, task 2 is to filter, task 3 is to organize,

2024-03-16 22:28:24.388 | INFO     | metagpt.utils.cost_manager:update_cost:52 - Total running cost: $0.008 | Max budget: $10.000 | Current cost: $0.008, prompt_tokens: 306, completion_tokens: 251
2024-03-16 22:28:24.389 | WARNING  | metagpt.utils.common:wrapper:571 - There is a exception in role's execution, in order to resume, we delete the newest role communication message in the role's memory.


 and task 4 is to save the final output.


Exception: Traceback (most recent call last):
  File "/root/workspace/StreamChatPlayground/.venv/lib/python3.10/site-packages/metagpt/utils/common.py", line 562, in wrapper
    return await func(self, *args, **kwargs)
  File "/root/workspace/StreamChatPlayground/.venv/lib/python3.10/site-packages/metagpt/roles/role.py", line 558, in run
    rsp = await self.react()
  File "/root/workspace/StreamChatPlayground/.venv/lib/python3.10/site-packages/metagpt/roles/role.py", line 529, in react
    rsp = await self._plan_and_act()
  File "/root/workspace/StreamChatPlayground/.venv/lib/python3.10/site-packages/metagpt/roles/role.py", line 489, in _plan_and_act
    await self.planner.update_plan(goal=goal)
  File "/root/workspace/StreamChatPlayground/.venv/lib/python3.10/site-packages/metagpt/strategy/planner.py", line 56, in update_plan
    rsp = await WritePlan().run(context, max_tasks=max_tasks, use_tools=self.use_tools)
  File "/root/workspace/StreamChatPlayground/.venv/lib/python3.10/site-pa